In [3]:
import shutil
import tkinter as tk
from tkinter import ttk, messagebox, filedialog

import json
import numpy as np
import matplotlib.pyplot as plt

from training import (
    read_tles_from_file,
    generate_benchmark_positions,
    generate_sequences_from_tles,
    load_trained_model,
    load_xgboost_model,
    predict_sequence,
    predict_xgboost_sequence,
    save_predictions_for_cesium_with_actual
)

def launch_predictor():
    root = tk.Tk()
    root.title("Satellite Predictor")

    num_var   = tk.StringVar()               # empty to start
    model_var = tk.StringVar(value="LSTM")   # default choice

    # ─── Layout ────────────────────────────────────────────────────────────────
    tk.Label(root, text="Number of Satellites:")\
      .grid(row=0, column=0, sticky="e", padx=5, pady=5)
    entry_num = tk.Entry(root, textvariable=num_var, width=10)
    entry_num.grid(row=0, column=1, padx=5, pady=5)
    entry_num.focus_set()

    tk.Label(root, text="Select Model:")\
      .grid(row=1, column=0, sticky="e", padx=5, pady=5)
    ttk.Combobox(
      root, textvariable=model_var,
      values=["LSTM","GRU","XGBoost"],
      state="readonly", width=8
    ).grid(row=1, column=1, padx=5, pady=5)

    # ─── Action ────────────────────────────────────────────────────────────────
    def on_start(event=None):
        # 1) parse N
        try:
            n = int(num_var.get())
        except ValueError:
            return messagebox.showerror("Input Error","Enter a valid integer")

        # 2) pick TLE file
        file_path = filedialog.askopenfilename(
            title="Select TLE file",
            filetypes=[("Text files","*.txt"),("All files","*.*")]
        )
        if not file_path:
            return messagebox.showwarning("No File","You must choose a TLE .txt")

        # 3) build exactly N sequences + benchmarks
        tles                = read_tles_from_file(file_path, num_samples=n)
        benchmark_positions = generate_benchmark_positions(
                                   tles=tles, num_points=30, step_sec=20
                               )
        sequences_all       = generate_sequences_from_tles(
                                   tles=tles, num_points=30, step_sec=20
                               )
        # ── QUICK FIX: slice off the 7th feature (ecc) so scaler sees 6 dims ─────
        sequences = sequences_all[..., :6]   # now shape (N,30,6)

        # 4) predict
        model_name = model_var.get()
        if model_name=="XGBoost":
            m, s   = load_xgboost_model("xgb")
            preds  = predict_xgboost_sequence(m, s, sequences)
        else:
            tag    = model_name.lower()
            m, s   = load_trained_model(tag)
            preds  = predict_sequence(m, s, sequences)

        # 5) write JSON + proxy for Cesium
        save_predictions_for_cesium_with_actual(
            preds, benchmark_positions, model_name=model_name
        )
        shutil.copy(f"predictions_{model_name}.json","predictions.json")

        # 6) notify
        messagebox.showinfo("Done",f"{model_name} predictions saved!")
        root.destroy()

        # 7) compute & plot errors side-by-side
        actual     = sequences[:len(preds), -1, 0:3]     # (N,3)
        errors_km  = np.linalg.norm(preds - actual, axis=1)
        radii      = np.linalg.norm(actual, axis=1)
        errors_pct = errors_km / radii * 100
        x = np.arange(len(preds))

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))
        ax1.stem(x, errors_pct,
                 linefmt='b-', markerfmt='o', basefmt=' ')
        ax1.set_title(f"{model_name} Prediction Error (%)")
        ax1.set_xlabel("Satellite Index")
        ax1.set_ylabel("Error (%)")

        ax2.stem(x, errors_km,
                 linefmt='r-', markerfmt='x', basefmt=' ')
        ax2.set_title(f"{model_name} Prediction Distance Missed (km)")
        ax2.set_xlabel("Satellite Index")
        ax2.set_ylabel("Distance Missed (km)")

        plt.tight_layout()
        plt.show()

    # ─── Bindings & Run ────────────────────────────────────────────────────────
    entry_num.bind("<Return>", on_start)
    root.bind("<Return>", on_start)
    root.bind("<Escape>", lambda e: root.destroy())
    tk.Button(root, text="Start Prediction", command=on_start)\
      .grid(row=2, column=0, columnspan=2, pady=10, ipadx=10)

    root.mainloop()

# ──────────────────────────────────────────────────────────────────────────────
# Launch the whole thing
# ──────────────────────────────────────────────────────────────────────────────
launch_predictor()


SGP4 error 1 for AEROCUBE 4.5B            at step 0
Generated 999 valid sequences out of 1000 TLEs.
SGP4 error 1 for AEROCUBE 4.5B            at step 0
Generated 999 valid sequences out of 1000 TLEs.


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cczap\AppData\Local\Temp\ipykernel_22100\3436116650.py", line 77, in on_start
    preds  = predict_sequence(m, s, sequences)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\cczap\Downloads\GradCapstone\training.py", line 201, in predict_sequence
    inv       = scaler.inverse_transform(padded)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cczap\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_data.py", line 581, in inverse_transform
    X -= self.min_
ValueError: operands could not be broadcast together with shapes (999,7) (6,) (999,7) 
